In [ ]:
HTML(read(open("style.css"), String))

# Iterative Deepening

In [ ]:
using Pkg
# Pkg.add("Chess")
using Chess
using Random
# Pkg.add("DataStructures")
using DataStructures
# Pkg.add("NBInclude")
using NBInclude

In [ ]:
@nbinclude("EvaluatePosition.ipynb")

In [ ]:
@nbinclude("Memoization.ipynb")

In [ ]:
@nbinclude("QuiescenceSearch.ipynb")

## maxValue

The `maxValue` function takes in 3 arguments and 2 optional arguments.
1. `State` is a chess `state` of type `Board`
1. `score` is the static centipawn evaluation of the `state`
1. `depth` is the number of halfmoves the engine should analyze before terminating 
1. `alpha` is optional and is default to -Infinity. Alpha is a minimal value that has been calculated during the recursive process
1. `beta`  is optional and is default to Infinity . Beta is a maximal value that has been calculated during the recursive process
        
The function returns the maximal centipawn evaluation of the current position for the player playing white where both players have played the optimal moves according to the algorithm and terminating after the given depth. All possible moves are sorted by a `PriorityQueue` using the evaluation of the position. Good moves will be prioritized which will increase the chance of pruning paths. 

In [ ]:
function maxValue(State::Board, score::Int64, hash::UInt64, depth::Int64,
                  cache::Dict{UInt64, Tuple{String, Int64, Int64}}, alpha::Int64=-200000, beta::Int64=200000)::Int64
    if isterminal(State) 
        return terminal_evaluation(State)
    end
    if depth == 0
        return score
        # return quiesceSee(State, score, alpha, beta)
        # return quiesce(State, score, hash, alpha, beta)
        # return evaluate(State, quiesce, score, hash, 0, alpha, beta)
    end
    value = alpha
    queue = PriorityQueue{Move, Int64}()
    for move in moves(State)
        undoinfo = domove!(State, move)
        val = value_cache(hash, depth-1, cache)
        undomove!(State, undoinfo)
        if val == nothing
            val = -200000
        end
        enqueue!(queue, move, -val)
    end
    while !isempty(queue)
        move = peek(queue)[1]
        nextEval, nextHash = updateBoardData(State, score, hash, move)
        undoinfo = domove!(State, move)
        value = max(value, evaluate(State, minValue, nextEval, nextHash, depth - 1, cache, value, beta))
        undomove!(State, undoinfo)
        if value >= beta
            return value
        end
        delete!(queue, move)
    end
    return value
end

## minValue

The `minValue` function takes in 3 arguments and 2 optional arguments.
1. `State` is a chess `state` of type `Board`
1. `score` is the static centipawn evaluation of the `state`
1. `depth` is the number of halfmoves the engine should analyze before terminating 
1. `alpha` is optional and is default to -Infinity. Alpha is a minimal value that has been calculated during the recursive process
1. `beta`  is optional and is default to Infinity . Beta is a maximal value that has been calculated during the recursive process
        
The function returns the maximal centipawn evaluation of the current position for the player playing white where both players have played the optimal moves according to the algorithm and terminating after the given depth. All possible moves are sorted by a `PriorityQueue` using the evaluation of the position. Good moves will be prioritized which will increase the chance of pruning paths. 

In [ ]:
function minValue(State::Board, score::Int64, hash::UInt64, depth::Int64,
                  cache::Dict{UInt64, Tuple{String, Int64, Int64}}, alpha::Int64=-200000, beta::Int64=200000)::Int64
    if isterminal(State)
        return terminal_evaluation(State)
    end
    if depth == 0
        return score
        # return quiesceSee(State, score, -beta, -alpha)
        # return -quiesce(State, score, hash, -beta, -alpha)
        # return -evaluate(State, quiesce, score, hash, 0, -beta, -alpha)
    end
    value = beta
    queue = PriorityQueue{Move, Int64}()
    for move in moves(State)
        undoinfo = domove!(State, move)
        val = value_cache(hash, depth-1, cache)
        undomove!(State, undoinfo)
        if val == nothing
            val = 200000
        end
        enqueue!(queue, move, val)
    end
    while !isempty(queue)
        move = peek(queue)[1]
        nextEval, nextHash = updateBoardData(State, score, hash, move)
        undoinfo = domove!(State, move)
        value = min(value, evaluate(State, maxValue, nextEval, nextHash, depth - 1, cache, alpha, value))
        undomove!(State, undoinfo)
        if value <= alpha
            return value
        end
        delete!(queue, move)
    end
    return value
end

The function `value_cache` is a helping function for the `minValue` and `maxValue` function. It takes in 2 arguments:
1. `State` is a chess `state` of type `Board`
1. `depth` is the number of halfmoves the engine should analyze before terminating

The function looks into the Cache and returns any previously saved values for this position. This information is used to sort good moves inside of the PriorityQueue.
The function returns the value for this `state` if the `state` is in the Cache and has a sufficient pre-calculated depth. If the cache does not have an entry for this `state` or entry does not have sufficient depth the function will return `nothing`.

In [ ]:
function value_cache(hash::UInt64, depth::Int64, cache::Dict{UInt64, Tuple{String, Int64, Int64}})
    if hash in keys(cache)
        _, value, d = cache[hash]
        if d >= depth
            return value
        end
    end
    # new move or no entry with enough depth
    return nothing
end

The `alphaBetaPruning` function takes in 3 arguments
1. `State` is the current state of type `Board`
1. `score` is the static centipawn evaluation of the static position
1. `depth` is the number of halfmoves the engine should analyze before terminating

The function returns the best value and the best move the moving player can play in the current position. It calls the `Iterative Deepening` algorithm. If multiple moves are found which result in the best evaluation a random move will be chosen.

In [ ]:
function pd_evaluate(State::Board, f::Function, score::Int64, hash::UInt64, depth::Int64, 
                     cache::Dict{UInt64, Tuple{String, Int64, Int64}})
    bestVal = score
    for d in 1:depth
        bestVal = evaluate(State, f, score, hash, d, cache)
        if bestVal in [-100000, 100000]
            return bestVal, d
        end
    end
    return bestVal, depth
end

In [ ]:
function iterativeDeepening(State::Board, score::Int64, hash::UInt64, depth::Int64, 
                            cache::Dict{UInt64, Tuple{String, Int64, Int64}} = initCache())
    side = sidetomove(State)
    bestVal, depth = (side == WHITE) ? pd_evaluate(State, maxValue, score, hash, depth, cache) : 
                                       pd_evaluate(State, minValue, score, hash, depth, cache)
    next_moves = moves(State)
    BestMoves::Array{Move} = []
    for move in next_moves
        nextHash=zobrist_hash(State,hash,move )
        
        if !(nextHash in keys(cache))
            continue
        end
        flag, val, d = cache[nextHash]
        if flag =="=" && val==bestVal 
            append!(BestMoves, [move])
        end    
    end
    
    BestMove::Move = rand(BestMoves)
    return bestVal, BestMove
end